In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
# import torch.nn.functional as F
import pandas as pd
import numpy as np
import json

import sys
import os

sys.path.append(os.path.abspath('../'))
from tool import *

data_root = '../../MyData/'

# Load data (With GICS_Sector)

In [2]:
total_embedding = obtain_total_embedding(data_root)
print(len(total_embedding))

# Check dataset
nan_proportion = total_embedding.isna().mean()
print(f"The nan value proportion in each column:\n{nan_proportion}\n\n")

Length of item1_embedding: 1197
Length of other_embedding: 1197
1197
The nan value proportion in each column:
cik                           0.000000
tic                           0.000000
Year                          0.000000
item1_embeddings              0.000000
GICS_Sector                   0.006683
SP_SHORT_DESC_embeddings      0.041771
SP_LONG_DESC_embeddings       0.095238
ORBIS_PROD_SERV_embeddings    0.208020
ORBIS_OVERVIEW_embeddings     0.168755
dtype: float64




# Load ae

In [3]:
from model.obtain_model import load_ae
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

trained_ae = load_ae("../model/saved_models/basic_ae.pth", device)

In [ ]:
# # Assuming x is a batch of input data (1D, not images)
# x = next(iter(test_loader))  # Get a batch of data
# x = x[0][8:16]

# # Encode the input
# trained_ae.eval()
# with torch.no_grad():
#     x = x.unsqueeze(1)
#     generated_samples = trained_ae(x.to(device))
    
#     # Move to CPU and convert to numpy if necessary
#     x = x.cpu()
#     x = x.squeeze(1)
#     print(x.shape)
#     generated_samples = generated_samples.cpu()
#     generated_samples = generated_samples.squeeze(1)
#     print(generated_samples.shape)
    
#     # Plot the original and generated samples in separate subplots
#     fig, axes = plt.subplots(8, 1, figsize=(12, 24))  # 8 rows, 1 column
    
#     for i in range(8):
#         axes[i].plot(x[i][900:1100], label="Original", linestyle='-', color='b', alpha=0.7)
#         axes[i].plot(generated_samples[i][900:1100], label="Generated", linestyle='--', color='r', alpha=0.7)
#         axes[i].set_title(f"Sample {i+1}")
#         axes[i].set_xlabel("Data Dimension")
#         axes[i].set_ylabel("Value")
#         axes[i].legend()
    
#     plt.tight_layout()
#     plt.show()


# Prepare data

In [4]:
target_list = ['item1_embeddings', 'SP_LONG_DESC_embeddings', 'SP_SHORT_DESC_embeddings', 'ORBIS_PROD_SERV_embeddings', 'ORBIS_OVERVIEW_embeddings']
info_list = ['cik', 'tic', 'Year', 'GICS_Sector']

exp_df = convert_to_array(total_embedding, info_list, target_list, 1536, False)
print(type(exp_df))
print(exp_df.shape)


# Dictionary to store the stacked embeddings as PyTorch tensors
embedding_tensors = {}

# Loop through the columns and convert to PyTorch tensor
for col in target_list:
    numpy_array = np.vstack(exp_df[col].values)  # Stack the column values
    embedding_tensors[col] = torch.tensor(numpy_array, dtype=torch.float32)  # Convert to tensor

# embedding_tensors is a dictionary containing all the openai embeddings I have.
emb_num = 0
for col in target_list:
    emb_num += len(embedding_tensors[col])
print(f"total number of embedding: {emb_num}")

<class 'pandas.core.frame.DataFrame'>
(1197, 9)
total number of embedding: 5985


In [8]:
latent_tensors = {}
for col in target_list:
    latent_tensors[col] = safe_inference(trained_ae.encoder_net, embedding_tensors[col].to(device))

print(latent_tensors)

emb_num = 0
for col in target_list:
    emb_num += len(latent_tensors[col])
print(f"total number of embedding: {emb_num}")

{'item1_embeddings': tensor([[-0.5898,  0.9802, -0.9664,  ...,  0.9971,  0.9972, -0.3560],
        [ 0.9821,  0.4754,  0.8698,  ...,  0.9983,  0.9933,  0.3218],
        [ 0.9946, -0.4087,  0.6804,  ...,  0.9944,  0.9053,  0.3607],
        ...,
        [-0.3125,  0.9592,  0.8852,  ...,  0.9983,  0.9903,  0.9140],
        [-0.6024,  0.8309,  0.6144,  ...,  0.9998,  0.8273, -0.8432],
        [ 0.5317,  0.9640,  0.9502,  ..., -0.9994, -0.1773,  0.8744]],
       device='cuda:0'), 'SP_LONG_DESC_embeddings': tensor([[-0.8353,  0.9734, -0.9823,  ...,  0.9971,  0.9976, -0.7578],
        [ 0.9717,  0.7132,  0.8791,  ...,  0.9911,  0.8676, -0.7440],
        [ 0.9719, -0.3351,  0.9141,  ...,  0.9956,  0.9862,  0.3395],
        ...,
        [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
        [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
        [-0.1758,  0.9904,  0.8948,  ..., -0.9995, -0.1103,  0.8007]],
       device='cuda:0'), 'SP_SHORT_DESC_embeddings': tensor

In [24]:
for col in target_list:
    exp_df[col] = latent_tensors[col].tolist()

In [7]:
# exp_df = pd.read_csv("./data/embedding_256.csv")
# exp_df